<a href="https://colab.research.google.com/github/ipietri/w266_Final_Project/blob/master/notebooks/TRAC-2-notebooks/grayscaling/TRAC_2_Track_B_Grayscaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TRAC-2 - Track B w/ Grayscaling
---
[Code Source](https://github.com/ainagari/scalar_adjs)

* [BERT Knows Punta Cana is not just beautiful, it’s gorgeous:
Ranking Scalar Adjectives with Contextualised Representations](https://aclanthology.org/2020.emnlp-main.598.pdf)\
*[Scalar Adjective Identification and Multilingual Ranking
](https://arxiv.org/abs/2105.01180)\
*[Identifying and Ordering Scalar Adjectives Using Lexical Substitution](https://www.proquest.com/openview/aade435a5bbdcf41e2b8c24e648826cc/1.pdf?pq-origsite=gscholar&cbl=18750)\
*[A Gold Standard for Scalar Adjectives](https://aclanthology.org/L16-1424/)


In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  path = r'/content/drive/MyDrive/w266'
except ModuleNotFoundError:
  path = r'data'

Mounted at /content/drive


<a id='section01'></a>
# Load Modules

In [2]:
#!pip install -U nltk
import nltk; nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
%%capture
!pip install -q transformers

In [4]:
%%capture
!pip install datasets
import datasets 
from datasets import load_dataset, Dataset, DatasetDict

In [5]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn import metrics
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
#from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import logging
logging.basicConfig(level=logging.ERROR)

In [6]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [7]:
from nltk.corpus import wordnet as wn
import gzip
import pickle
import numpy as np
import sys
from scipy.spatial.distance import cosine
from operator import itemgetter
from collections import defaultdict
#from pymagnitude import *
import argparse

import itertools

In [8]:
# import eda script from github
!git clone https://github.com/ainagari/scalar_adjs

Cloning into 'scalar_adjs'...
remote: Enumerating objects: 854, done.
remote: Counting objects: 100% (438/438), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 854 (delta 91), reused 358 (delta 45), pack-reused 416
Receiving objects: 100% (854/854), 13.47 MiB | 17.68 MiB/s, done.
Resolving deltas: 100% (129/129), done.


In [9]:
# import fucntions from scalar_adjs
import sys

# sys.path is a list of absolute path strings
sys.path.append('/content/scalar_adjs/')

from read_scalar_datasets import read_scales


In [10]:
!ls

drive  sample_data  scalar_adjs


In [11]:
# get the greyscaling augmentation script from our repo
!git clone https://github.com/ipietri/w266_Final_Project
#sys.path.append('/content/')

Cloning into 'w266_Final_Project'...
fatal: could not read Username for 'https://github.com': No such device or address


<a id='section02'></a>
# Import and Preprocess Data

In [12]:
trac2_task_b = pd.read_csv('/content/drive/MyDrive/w266/task_B_data_oversampled.csv')
print('TASK B: ',trac2_task_b.shape)

trac2_dev = pd.read_csv('/content/drive/MyDrive/w266/trac2_eng_dev.csv')
print('dev: ',trac2_dev.shape)


TASK B:  (7908, 2)
dev:  (1066, 4)


In [13]:
# create a dev dataset for track a and one for track b 
trac2_task_b_dev= trac2_dev[['Text','Sub-task B']]

In [14]:

print("TASK B unique sentiments: ", trac2_task_b['Sub-task B'].unique())

TASK B unique sentiments:  ['NGEN' 'GEN']


In [15]:
# rename columns
trac2_task_b.rename(columns = {'Text':'text', 'Sub-task B': 'label'}, inplace = True)
trac2_task_b_dev.rename(columns = {'Text':'text', 'Sub-task B': 'label'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
# remove NaNs and return without missing values in response_text
# train
trac2_task_b.dropna(subset = ['text'], inplace=True)

# dev

trac2_task_b_dev.dropna(subset = ['text'], inplace=True)

print('Train: ', trac2_task_b.shape)
print('Dev: ', trac2_task_b_dev.shape)

Train:  (7908, 2)
Dev:  (1066, 2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [17]:
# Dummy variables

# task B
task_b_labels = {'NGEN':0, 'GEN': 1}
trac2_task_b['label'] = trac2_task_b['label'].map(task_b_labels).astype(int)
trac2_task_b_dev['label'] = trac2_task_b_dev['label'].map(task_b_labels).astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Greyscale
Adapted from Scalar Adj Code [Extract Relevant Text](https://github.com/ainagari/scalar_adjs/blob/master/extract_flickr_scalar.py)



In [18]:
import pickle
import pdb
import spacy
import os
import collections

nlp = spacy.load("en_core_web_sm")

language_str = "en" #set to english -- one dataset in English only

### Identify the location of every word present the three types of scales
Extract and save out as a dictionary for every example that contains at least one scaled word. Adapted from extract_flickr_scalar.py

In [19]:
rankings = dict()
datanames = ["demelo", "crowd", "wilkinson"]

import dill
filename = "/content/drive/MyDrive/w266/scales_with_milder_option.pickle"
scales_with_milder_option = dill.load(open(filename, "rb"))

In [20]:
rankings = dict()
datanames = ["demelo", "crowd", "wilkinson"]
for dataname in datanames:
    r = read_scales("/content/scalar_adjs/data/" + dataname + "/gold_rankings/")
    rankings[dataname] = r

my_words = set()
for dataname in rankings:
    for scale in rankings[dataname]:
        for word in rankings[dataname][scale]:
            words = word.split(" || ")
            for w in words:
                my_words.add(w)

word_sentence_dict = dict()
for word in my_words:
    word_sentence_dict[word] = set()

def accepted_pos(pos):
    if pos in ["ADJ","ADV", "ADP","VERB","DET"]:
        return True
    return False


## Identify the position of the word which exists in any of the scales

  Since there are ties in our scales
  the milder word may be one or more words.
  So if the original word is foo || bar and the 
  milder word is foolish || barish this 
  {foo: foolish, foo: barish, bar: foolish,  bar: barish}

In [21]:
# create a nested dictionary with every scale, scale list with equalities, and all words in the scale

import collections
scales_dict = collections.defaultdict(dict)

for dataname in datanames:
  for scale_file_name, scale in rankings[dataname].items():
    words_in_scale = []
    for ws in scale:
      # split if there are ties
      words_in_scale.extend(ws.split(" || "))
    scales_dict[dataname][str(scale)] = tuple(words_in_scale)

In [22]:
def locate_scale_word(test_col, label_col):
  '''
  return a column with any scale_words 
  in the text column and their position
  '''
  more_test = collections.defaultdict(lambda: collections.defaultdict(dict))
  for data_name in datanames:
      for scale_name, words in scales_dict[data_name].items():
        for word in words:
          # convert text into a list of words to avoid partial matches found using .find()
          
          sentence_words = test_col.replace("'", "") 
          sentence_words = sentence_words.lower().split(" ")
          
          if word in sentence_words:
            pos = sentence_words.index(word)
         #   more_test[data_name]['label'] = label_col
            # assume only one word to be replaced
            more_test[data_name][word]['position'] = int(pos)
          #  more_test[data_name]['sentence'][word] = test_col
            more_test[data_name][word]['milder_words'] = scales_with_milder_option[data_name][word]
            

  return more_test

In [23]:
# convert relevant df columns to dictionary
# task B
trac2_task_b['new_col'] = trac2_task_b.apply(lambda x: locate_scale_word(x['text'], x['label']), axis = 1)
dict_from_df_b = trac2_task_b.to_dict('index')


# Augment

In [24]:
%cd "/content/drive/MyDrive/w266"
%run grey_scale_augmentation.ipynb

/content/drive/MyDrive/w266


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [25]:
task_b = augment_greyscaling(dict_from_df_b, datanames, 'text', 'label')

# append the augmented data plus labels

new_list = list(zip(labels_list, augmented_text_list))
new_df = pd.DataFrame(new_list, columns = ['label', 'text'])
print("Number of augmented examples: ", len(new_df))

# append to the original examples 
trac2_task_b_aug = new_df.append(trac2_task_b)

# save out greyscale adjusted dataset
trac2_task_b_aug.to_csv('/content/drive/MyDrive/w266/grey_scaled_augmented_oversampled_subtask_b_train_data.csv', index=False)
print("Total number of examples: ", len(trac2_task_b_aug))

Number of augmented examples:  7415
Total number of examples:  15323


# Convert to Hugging Face friendly format

In [26]:
trac2_task_b_aug = pd.read_csv('/content/drive/MyDrive/w266/grey_scaled_augmented_oversampled_subtask_b_train_data.csv')
trac2_task_b_aug = trac2_task_b_aug.drop(['new_col'], axis = 1)


In [27]:
# change to dataset to work with Huggingface transformer & remove unused columns

train_dataset = Dataset.from_pandas(trac2_task_b_aug)
dev_dataset = Dataset.from_pandas(trac2_task_b_dev)
dev_dataset = dev_dataset.remove_columns(column_names= '__index_level_0__')

# combine into a DataDictionary for huggingface use
dataset = DatasetDict({
    'train': train_dataset,
    'dev': dev_dataset 
})


dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 15323
    })
    dev: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

# Tokenize

In [29]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
max_length = 150
tokenizer = AutoTokenizer.from_pretrained(model_name, max_length = int(max_length))

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)    

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Model

In [30]:
from transformers import AutoModelForSequenceClassification
num_labels = 2
epochs = 2
iterations = 5
model = (AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device))
dataset_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [32]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1_weighted = f1_score(labels, preds, average="weighted")
    f1_macro = f1_score(labels, preds, average = 'macro')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1_weighted, "f1_macro": f1_macro} 

In [33]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(dataset_encoded["train"]) // batch_size
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=epochs,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                 # metric_for_best_model="f1_macro",
                                 # weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  disable_tqdm=False
                                  )

In [35]:
# category mapping ['NGEN':0  'GEN': 1]
from sklearn.metrics import classification_report

accuracy_list = []
weighted_f1_score_list = []
macro_f1_score_list = []
NGEN_f1_score = []
GEN_f1_score = []

for i in range(iterations):
  try:
    del trainer
    del results
    del cr
  except: pass


  trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=dataset_encoded["train"],
                  eval_dataset=dataset_encoded["dev"])
  trainer.train()
  results = trainer.evaluate()

  # append macro metrics to lists
  accuracy_list.append(results.get('eval_accuracy'))
  weighted_f1_score_list.append(results.get("eval_f1"))
  macro_f1_score_list.append(results.get("eval_f1_macro"))

  trainer.predict(dataset_encoded["dev"])
  # append the class-level F1 scores
  outputs = trainer.predict(dataset_encoded["dev"])
  predictions = outputs.predictions.argmax(1)
  labels = dataset_encoded["dev"]['label']
  cr = classification_report(labels, predictions, digits=3, output_dict=True)
  NGEN_f1_score.append(cr.get('0').get("f1-score"))
  GEN_f1_score.append(cr.get('1').get("f1-score"))
  
  print(f'---------------------------Iteration {i+1} Complete---------------------------\n')

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 15323
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3832


Epoch,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
1,0.015700,0.538439,0.937148,0.932745,0.719512
2,0.004800,0.591674,0.942777,0.936669,0.728546


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1066
  Batch size = 8
Saving model checkpoint to results/checkpoint-1916
Configuration saved in results/checkpoint-1916/config.json
Model weights saved in results/checkpoint-1916/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1066
  Batch size = 8
Saving model checkpoint to results/checkpoint-3832
Configuration saved in results/checkpoint-3832/config.json
Model weights saved in results/checkpoint-3832/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-1916 (score: 0.5384387373924255).
The following columns in the evaluation s

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8
The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 15323
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3832


---------------------------Iteration 1 Complete---------------------------



...

---------------------------Iteration 4 Complete---------------------------



Epoch,Training Loss,Validation Loss,Accuracy,F1,F1 Macro
1,0.003500,0.600464,0.933396,0.934415,0.746980
2,0.001900,0.669100,0.944653,0.937024,0.724327


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1066
  Batch size = 8
Saving model checkpoint to results/checkpoint-1916
Configuration saved in results/checkpoint-1916/config.json
Model weights saved in results/checkpoint-1916/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1066
  Batch size = 8
Saving model checkpoint to results/checkpoint-3832
Configuration saved in results/checkpoint-3832/config.json
Model weights saved in results/checkpoint-3832/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-1916 (score: 0.6004644632339478).
The following columns in the evaluation s

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


---------------------------Iteration 5 Complete---------------------------



# Evaluate

In [36]:
import statistics

print("%15s %s (%s)" % ("","Mean", "StDev"))

print("-"*29)
print("Macro Scores")
print("-"*29)

print(f"%15s %s (%s)" %("Accuracy",
    round(statistics.mean(accuracy_list),3),
    round(statistics.stdev(accuracy_list),3)))
print(f"%15s %5s (%s)" %("Macro F1",
    round(statistics.mean(macro_f1_score_list),3),
    round(statistics.stdev(macro_f1_score_list),3)))
print(f"%15s %5s (%s)" %("Weighted F1",
    round(statistics.mean(weighted_f1_score_list),3),
    round(statistics.stdev(weighted_f1_score_list),3)))

print("-"*29)
print("Class Scores")
print("-"*29)

print(f"%15s %s (%s)" %("NGEN",
    round(statistics.mean(NGEN_f1_score),3),
    round(statistics.stdev(NGEN_f1_score),3)))
print(f"%15s %5s (%s)" %("GEN",
    round(statistics.mean(GEN_f1_score),3),
    round(statistics.stdev(GEN_f1_score),3)))

                Mean (StDev)
-----------------------------
Macro Scores
-----------------------------
       Accuracy 0.94 (0.005)
       Macro F1 0.724 (0.014)
    Weighted F1 0.935 (0.002)
-----------------------------
Class Scores
-----------------------------
           NGEN 0.968 (0.003)
            GEN  0.48 (0.031)
